# Dependencies

In [25]:
# %load_ext autoreload
# %autoreload 2
# import nltk
# import ipywidgets
# import pandas as pd
# import numpy as np
# import time
# from tqdm import tqdm
# tqdm.pandas()
# # from pandarallel import pandarallel
# # pandarallel.initialize(progress_bar=True)
# import torch
# import datetime
# import os
import datasets_handler
import evaluation_metrics
import zeroberto
from zeroberto import ZeroBERTo
# # import datasets
# # from datasets import Dataset
# from tqdm import tqdm
# from sklearn.preprocessing import LabelEncoder
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
import gc

# Dataset

In [8]:
which_dataset = 'folhauol' 
# which_dataset = 'bbc-news'
# which_dataset = 'ag_news'
# which_dataset = 'imdb'

# hyp_template = "{}"
# hyp_template = "Esse texto está relacionado a {}."
hyp_template = "O tema principal deste texto é {}."
# hyp_template = "this text is about {}."
# hyp_template = "this article is about {}."

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)
classes_list = list(raw_data[class_col].unique())
print(classes_list)

['Poder e Política no Brasil', 'Mercado', 'Notícias de fora do Brasil', 'Esporte', 'Tecnologia', 'Meio Ambiente', 'Equilíbrio e Saúde', 'Educação', 'TV, Televisão e Entretenimento', 'Ciência', 'Turismo', 'Comida']


# ZeroBERTo

## Parameters

In [9]:
# max_inferences = len(train) #estava em 5000
max_inferences = 5000

# zeroshot_method = "probability_threshold"
# probability_goal = 0.9
# top_n_goal = 8

zeroshot_method = "dotproduct"  
# zeroshot_method = "kmeans"  

zeroshot_config = {
    'dataset':which_dataset,
    'class_col':class_col,
    'data_col':data_col,
    'split':"zeroshot",
    'method':zeroshot_method,
    # 'prob_goal':probability_goal,
    # 'top_n_goal':top_n_goal,
    'max_inferences':max_inferences,
    'classes':classes_list,#list(dict_classes_folha.values())
    'template': hyp_template,
    'random_state':422,
    # 'trainings_done':0
}
zeroshot_config

{'dataset': 'folhauol',
 'class_col': 'class',
 'data_col': 'text',
 'split': 'zeroshot',
 'method': 'dotproduct',
 'max_inferences': 5000,
 'classes': ['Poder e Política no Brasil',
  'Mercado',
  'Notícias de fora do Brasil',
  'Esporte',
  'Tecnologia',
  'Meio Ambiente',
  'Equilíbrio e Saúde',
  'Educação',
  'TV, Televisão e Entretenimento',
  'Ciência',
  'Turismo',
  'Comida'],
 'template': 'O tema principal deste texto é {}.',
 'random_state': 422}

In [10]:
train_data = raw_data.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state']).sort_index()
len(train_data)

5000

## Model

In [11]:
zeroshot_config['similarity_model'] = 'sentence-transformers/stsb-xlm-r-multilingual'
zeroshot_config['setfit_model'] = 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2'

model = ZeroBERTo(classes_list=zeroshot_config['classes'],hypothesis_template=zeroshot_config['template'],
                  train_dataset=train_data,labeling_method=zeroshot_config['method'],embeddingModel=zeroshot_config['similarity_model'])

In [12]:
model.embeddingModel

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

## Inference

In [13]:
# results = zeroberto.runZeroberto(model,train_data[data_col].to_list(),zeroshot_config)  ##X## COMENTAR AQUI 

Preds: 100  - Total time: 7.94 seconds - ETA: 6.6 minutes
Preds: 200  - Total time: 16.11 seconds - ETA: 6.7 minutes
Preds: 300  - Total time: 23.51 seconds - ETA: 6.5 minutes
Preds: 400  - Total time: 31.09 seconds - ETA: 6.5 minutes
Preds: 500  - Total time: 38.7 seconds - ETA: 6.4 minutes
Preds: 600  - Total time: 46.07 seconds - ETA: 6.4 minutes
Preds: 700  - Total time: 53.23 seconds - ETA: 6.3 minutes
Preds: 800  - Total time: 61.41 seconds - ETA: 6.4 minutes
Preds: 900  - Total time: 68.71 seconds - ETA: 6.4 minutes
Preds: 1000  - Total time: 75.71 seconds - ETA: 6.3 minutes
Preds: 1100  - Total time: 82.8 seconds - ETA: 6.3 minutes
Preds: 1200  - Total time: 90.67 seconds - ETA: 6.3 minutes
Preds: 1300  - Total time: 98.3 seconds - ETA: 6.3 minutes
Preds: 1400  - Total time: 105.39 seconds - ETA: 6.3 minutes
Preds: 1500  - Total time: 112.5 seconds - ETA: 6.3 minutes
Preds: 1600  - Total time: 119.47 seconds - ETA: 6.2 minutes
Preds: 1700  - Total time: 126.64 seconds - ETA: 6.

In [ ]:
results = model.runLabeling(train_data[data_col].to_list())

## Evaluation

In [14]:
df_results = model.evaluateLabeling(results)

top 1: {'accuracy': 0.5}
top 2: {'accuracy': 0.5}
top 3: {'accuracy': 0.4722222222222222}
top 4: {'accuracy': 0.5}
top 5: {'accuracy': 0.48333333333333334}
top 6: {'accuracy': 0.4722222222222222}
top 7: {'accuracy': 0.44047619047619047}
top 8: {'accuracy': 0.46875}
top 9: {'accuracy': 0.46296296296296297}
top 10: {'accuracy': 0.475}
top 11: {'accuracy': 0.48484848484848486}
top 12: {'accuracy': 0.4930555555555556}
top 13: {'accuracy': 0.48717948717948717}
top 14: {'accuracy': 0.4880952380952381}
top 15: {'accuracy': 0.4777777777777778}
top 16: {'accuracy': 0.4635416666666667}
top 5000: {'accuracy': 0.3246}


In [15]:
try: print(zeroshot_metrics)
except: pass
zeroshot_metrics = evaluation_metrics.get_metrics(df_results['prediction_code'].to_list(),df_results['class_code'].to_list())
print(zeroshot_metrics)

{'weighted': [{'accuracy': 0.3246}, {'precision': 0.47766436597703094}, {'recall': 0.3246}, {'f1': 0.32373635274282675}], 'macro': [{'accuracy': 0.3246}, {'precision': 0.3118963604155802}, {'recall': 0.32960798201063995}, {'f1': 0.2761013033975229}]}


## Save Labeling Results

In [16]:
save_time = evaluation_metrics.saveZeroshotResults(zeroshot_config,df_results)

predictions_and_probabilities_test_2023_04_07__20_13_57.csv
zeroshot_config_test_2023_04_07__20_13_57.csv


# Training

### Training Parameters

In [17]:
use_zeroshot_previous_step = True
n = 8
top_n = 8

batch_size = 4
num_text_pairs = 10
num_epochs = 1

zeroshot_config['top_n'] = top_n
zeroshot_config['n_examples'] = n
zeroshot_config['exec_time'] = save_time

zeroshot_config["batch_size"] = batch_size
zeroshot_config["num_pairs"] = num_text_pairs
zeroshot_config["num_epochs"] = num_epochs

zeroshot_config['st_train_epochs'] = 10
zeroshot_config['st_train_batch_size'] = 20

split = "zeroshot" if use_zeroshot_previous_step else "fewshot"

# Unfreeze the head and freeze the body -> head-only training
zeroshot_config['keep_body_frozen_setfit'] = True
# Unfreeze the head and unfreeze the body -> end-to-end training
# zeroshot_config['keep_body_frozen_setfit'] = False



### Previous data from labeling step

In [18]:
zeroshot_previous_data = datasets_handler.getZeroshotPreviousData(which_dataset,class_col,top_n=top_n,exec_time=zeroshot_config['exec_time'])
raw_data_final, zeroshot_config['new_class_col'] = datasets_handler.mergeLabelingToDataset(raw_data,zeroshot_previous_data,class_col)

In [19]:
df_train, df_test = datasets_handler.splitDataset(raw_data_final,zeroshot_config)
train_dataset,test_dataset = datasets_handler.buildDatasetDict(df_train,df_test)
train_documents = datasets_handler.splitDocuments(df_train[data_col])
# len(train_documents)

416


In [20]:
# df_test

## Treinamento do Corpo do Modelo -- não descomentar

In [21]:
# model.fit(train_documents,batch_size=zeroshot_config['st_train_batch_size'],epochs=zeroshot_config['st_train_epochs'])

# Fine-Tuning (SetFit)

## Model

In [22]:
# setfit_model.model_body = model.embeddingModel

setfit_model = SetFitModel.from_pretrained(zeroshot_config['setfit_model'],
                                    use_differentiable_head=True,
                                    head_params={"out_features":len(zeroshot_config['classes'])}
                                    )

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [23]:
# setfit_model.model_head.temperature = 1.0

In [26]:
# Create trainer
trainer = SetFitTrainer(
    model=setfit_model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=zeroshot_config["batch_size"],
    num_iterations=zeroshot_config["num_pairs"], # Number of text pairs to generate for contrastive learning
    num_epochs=zeroshot_config["num_epochs"], # Number of epochs to use for contrastive learning
    column_mapping = {data_col: "text", 'class_code': "label"} # NÃO mudar
)

In [27]:
%%time
# Train and evaluate
trainer.freeze() # Freeze the head
# trainer.train() # Train only the body
print(zeroshot_config)

trainer.unfreeze(keep_body_frozen=zeroshot_config['keep_body_frozen_setfit'])

trainer.train(
    num_epochs=zeroshot_config["num_epochs"], # The number of epochs to train the head or the whole model (body and head)
    batch_size=zeroshot_config["batch_size"],
    body_learning_rate=1e-5, # The body's learning rate
    learning_rate=1e-2, # The head's learning rate
    l2_weight=0.1, # Weight decay on **both** the body and head. If `None`, will use 0.01.
)
# zeroshot_config['trainings_done'] += 1

Applying column mapping to training dataset
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 512.


{'dataset': 'folhauol', 'class_col': 'class', 'data_col': 'text', 'split': 'zeroshot', 'method': 'dotproduct', 'max_inferences': 5000, 'classes': ['Poder e Política no Brasil', 'Mercado', 'Notícias de fora do Brasil', 'Esporte', 'Tecnologia', 'Meio Ambiente', 'Equilíbrio e Saúde', 'Educação', 'TV, Televisão e Entretenimento', 'Ciência', 'Turismo', 'Comida'], 'template': 'O tema principal deste texto é {}.', 'random_state': 422, 'similarity_model': 'sentence-transformers/stsb-xlm-r-multilingual', 'setfit_model': 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2', 'top_n': 8, 'n_examples': 8, 'exec_time': '2023_04_07__20_13_57', 'batch_size': 4, 'num_pairs': 10, 'num_epochs': 1, 'st_train_epochs': 10, 'st_train_batch_size': 20, 'keep_body_frozen_setfit': True, 'new_class_col': 'new_class'}


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 1min 30s, sys: 13.3 s, total: 1min 44s
Wall time: 45.6 s


In [28]:
gc.collect()

y_pred = zeroberto.getPredictions(trainer)
# 5000 sentences --- 4min40s ---sts-xlm-rr
# 5000 sentences ---  ---ricardo-filho/bert-portuguese

Running predictions on 4992 sentences.


## Evaluation

In [30]:
try: print(setfit_all_metrics)
except: pass
setfit_all_metrics = evaluation_metrics.get_metrics(y_pred ,test_dataset["class_code"])
print(setfit_all_metrics)

{'weighted': [{'accuracy': 0.08273237179487179}, {'precision': 0.007087451949478309}, {'recall': 0.08273237179487179}, {'f1': 0.013056398583712698}], 'macro': [{'accuracy': 0.08273237179487179}, {'precision': 0.007087451949478309}, {'recall': 0.08273237179487179}, {'f1': 0.013056398583712698}]}


/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'weighted': [{'accuracy': 0.08273237179487179}, {'precision': 0.007087451949478309}, {'recall': 0.08273237179487179}, {'f1': 0.013056398583712698}], 'macro': [{'accuracy': 0.08273237179487179}, {'precision': 0.007087451949478309}, {'recall': 0.08273237179487179}, {'f1': 0.013056398583712698}]}


In [ ]:
# new_embeddingModel = trainer.model.model_body

In [35]:
# list(zip(y_pred ,test_dataset["class_code"]))
y_pred==test_dataset["class_code"]

True

## Save CL results

In [ ]:
setfit_exec_time  = evaluation_metrics.saveResults(zeroshot_config,setfit_all_metrics)
print(zeroshot_config)

metrics_setfit_2023_04_07__18_52_03.csv
config_setfit_2023_04_07__18_52_03.csv
{'dataset': 'folhauol', 'class_col': 'class', 'data_col': 'text', 'split': 'zeroshot', 'method': 'dotproduct', 'max_inferences': 5000, 'classes': ['Poder e Política no Brasil', 'Mercado', 'Notícias de fora do Brasil', 'Esporte', 'Tecnologia', 'Meio Ambiente', 'Equilíbrio e Saúde', 'Educação', 'TV, Televisão e Entretenimento', 'Ciência', 'Turismo', 'Comida'], 'template': 'Esse texto está relacionado a {}.', 'random_state': 422, 'similarity_model': 'sentence-transformers/stsb-xlm-r-multilingual', 'setfit_model': 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2', 'top_n': 4, 'n_examples': 4, 'exec_time': '2023_04_07__18_02_19', 'batch_size': 4, 'num_pairs': 10, 'num_epochs': 1, 'st_train_epochs': 10, 'st_train_batch_size': 20, 'new_class_col': 'new_class', 'keep_body_frozen_setfit': True}


## Save Model

In [ ]:
# model_path = "/Users/alealcoforado/Documents/Projetos/Modelos/{exec}".format(exec=setfit_exec_time)
# print(model_path)
# trainer.model._save_pretrained(save_directory=model_path)

/Users/alealcoforado/Documents/Projetos/Modelos/2023_03_09__14_05_21
